## 1. 問題分析結果

### 競技プログラミング視点
- **原始根(Primitive Root)** の定義: 素数pに対して、g^1, g^2, ..., g^(p-1) (mod p) がすべて異なる値になる整数g
- 制約: p ≤ 10^9 の素数
- 必要な出力: 最小の原始根と原始根の総数

### 数学的考察
1. 素数pの原始根の個数は **オイラーのトーシェント関数 φ(p-1)** に等しい
2. gが原始根 ⇔ g^((p-1)/q) ≢ 1 (mod p) for all prime divisors q of (p-1)
3. 最小原始根を見つけるには、2から順に確認

### Python特有最適化ポイント
- `pow(base, exp, mod)` の組み込み関数を使用（高速な累乗剰余演算）
- 素因数分解の効率化
- オイラーのトーシェント関数の効率的計算

## 2. アルゴリズム比較表

| アプローチ | 時間計算量 | 空間計算量 | Python実装コスト | 可読性 | 備考 |
|---------|---------|---------|--------------|-------|------|
| 全探索(全gをチェック) | O(p² log p) | O(1) | 低 | ★★★ | 制約上TLE |
| 素因数分解活用 | O(√(p-1) + k·d·log p) | O(d) | 中 | ★★☆ | k:候補数, d:素因数個数 |

## 3. 採用アルゴリズム

**素因数分解を利用した原始根判定**
- (p-1)の素因数を求める
- 各候補gについて、g^((p-1)/q) ≢ 1 (mod p) を全素因数qで確認
- φ(p-1)で総数を計算

## 4. 実装

In [ ]:
import math
import os
import random
import re
import sys

from typing import List, Set

def prime_factors(n: int) -> List[int]:
    """
    nの素因数をリストで返す（重複なし）
    
    Time Complexity: O(√n)
    """
    factors = []
    # 2で割り切れる場合
    if n % 2 == 0:
        factors.append(2)
        while n % 2 == 0:
            n //= 2
    
    # 3以降の奇数でチェック
    i = 3
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 2
    
    # nが1より大きければ、それ自体が素数
    if n > 1:
        factors.append(n)
    
    return factors

def is_primitive_root(g: int, p: int, prime_divisors: List[int]) -> bool:
    """
    gがpの原始根かどうかを判定
    
    Args:
        g: 判定対象の整数
        p: 素数
        prime_divisors: (p-1)の素因数リスト
    
    Returns:
        gが原始根ならTrue
    
    Time Complexity: O(d·log p) where d = len(prime_divisors)
    """
    phi = p - 1
    
    # 各素因数qについて、g^((p-1)/q) ≢ 1 (mod p) を確認
    for q in prime_divisors:
        if pow(g, phi // q, p) == 1:
            return False
    
    return True

def euler_phi(n: int) -> int:
    """
    オイラーのトーシェント関数 φ(n) を計算
    
    Time Complexity: O(√n)
    """
    result = n
    
    # 2で割り切れる場合
    if n % 2 == 0:
        while n % 2 == 0:
            n //= 2
        result -= result // 2
    
    # 3以降の奇数でチェック
    p = 3
    while p * p <= n:
        if n % p == 0:
            while n % p == 0:
                n //= p
            result -= result // p
        p += 2
    
    if n > 1:
        result -= result // n
    
    return result

def solve_competitive(p: int) -> tuple:
    """
    競技プログラミング向け実装
    
    Args:
        p: 素数
    
    Returns:
        (最小原始根, 原始根の総数)
    
    Time Complexity: O(√p + k·d·log p) 
        where k = 最小原始根の値, d = (p-1)の素因数の個数
    Space Complexity: O(d)
    """
    # 特異ケース: p=2
    if p == 2:
        return 1, 1
    
    # (p-1)の素因数を求める
    prime_divisors = prime_factors(p - 1)
    
    # 最小原始根を探索
    smallest_root = 0
    for g in range(2, p):
        if is_primitive_root(g, p, prime_divisors):
            smallest_root = g
            break
    
    # 原始根の総数 = φ(p-1)
    total_count = euler_phi(p - 1)
    
    return smallest_root, total_count

if __name__ == '__main__':
    p = int(input().strip())
    # テスト用入力
    # p = 7
    
    smallest, total = solve_competitive(p)
    print(f"{smallest} {total}")

## 5. 検証

### サンプル入力0: p = 7
- p-1 = 6 の素因数: [2, 3]
- g=2: 2^3 mod 7 = 1 → 原始根でない
- g=3: 3^3 mod 7 = 6, 3^2 mod 7 = 2 → 原始根 ✓
- φ(6) = 2
- 出力: `3 2` ✓

### 計算量分析
- 素因数分解: O(√p)
- 原始根判定: 最悪でもO(p·d·log p)だが、実際は最小原始根は小さいことが多い
- φ(p-1)の計算: O(√p)
- 全体: p ≤ 10^9 でも実用的な時間で動作

# 原始根（Primitive Root）の解説

この文書は、**Jupyter Notebook などで数式が正しく LaTeX レンダリングされる**ように修正したマークダウン版です。数式は `$...$` または `$$...$$` を用いて記述しています。

---

## 1. 直感的な説明：原始根とは何か

**原始根**とは、

> 「ある数を何乗もしていくことで、剰余の世界にある全ての要素を生成できる数」

のことです。

---

## 2. 具体例：mod 7 の場合

$7$ を法とする剰余を考えます。$0$ を除くと、

$$
1,2,3,4,5,6
$$

の $6$ 個があります。

### 2 を使った場合

$$
\begin{aligned}
2^1 &\equiv 2 \pmod{7} \
2^2 &\equiv 4 \pmod{7} \
2^3 &\equiv 1 \pmod{7}
\end{aligned}
$$

ここで $1$ に戻ってしまい、${1,2,4}$ しか生成できません。

### 3 を使った場合

$$
\begin{aligned}
3^1 &\equiv 3 \pmod{7} \
3^2 &\equiv 2 \pmod{7} \
3^3 &\equiv 6 \pmod{7} \
3^4 &\equiv 4 \pmod{7} \
3^5 &\equiv 5 \pmod{7} \
3^6 &\equiv 1 \pmod{7}
\end{aligned}
$$

$1$ から $6$ まで全てが現れました。したがって **$3$ は mod $7$ の原始根**です。

---

## 3. 数学的な定義

正の整数 $n$ に対し、

$$
(\mathbb{Z}/n\mathbb{Z})^\times
$$

（$n$ と互いに素な整数の剰余類全体）を考えます。

この集合において、

> 自身の冪で全ての要素を生成できる元

を **原始根**と呼びます。

---

## 4. 位数（order）の定義

原始根を理解する鍵となる概念が **位数**です。

### 位数の定義

$$
\operatorname{ord}_n(a)
= \min{k > 0 \mid a^k \equiv 1 \pmod{n}}
$$

これは「$a^k$ が初めて $1$ に戻るまでの最小の周期」を表します。

---

## 5. 原始根の条件

オイラーのトーシェント関数 $\varphi(n)$ を用いると、

$$
\operatorname{ord}_n(g) = \varphi(n)
$$

を満たす $g$ が **原始根**です。

つまり、

> 位数が最大の元

が原始根になります。

---

## 6. 群論的な背景（直感的説明）

$p$ を素数とすると、

$$
(\mathbb{Z}/p\mathbb{Z})^\times
$$

は位数 $p-1$ の **巡回群**になります。巡回群には必ず生成元が存在し、その生成元が原始根です。

直感的には、

* 歩幅が合わないと一部しか回れない
* 歩幅がちょうど良いと全体を一周できる

というイメージです。

---

## 7. 原始根が存在する整数 $n$

原始根が存在する $n$ は、次の場合に限られます：

$$
n = 1,2,4,p^k,2p^k
$$

ただし $p$ は奇素数、$k \ge 1$ です。

例：

* $7$（素数） → 存在する
* $9 = 3^2$ → 存在する
* $14 = 2 \times 7$ → 存在する
* $8$ → 存在しない

---

## 8. 応用：暗号理論との関係

原始根 $g$ が存在すると、

$$
g^x \equiv y \pmod{p}
$$

という形の **離散対数問題**を定義できます。この問題は計算的に非常に困難であり、

* Diffie–Hellman 鍵共有
* ElGamal 暗号

などの基盤になっています。

---

## 9. まとめ

**原始根**とは、剰余乗法群を一つの元の冪によって完全に生成できる特別な数であり、その位数はトーシェント関数に等しくなります。理論的にも計算機科学的にも重要な概念です。